# Testing Captum  + Text Generation

This notebook was created to test the BERTViz package for attention visualization.

### Tested Model
- GPT-2
- GODEL
- Mistral 7B Instruct
- LlaMa 2 7B Chat (HF Version)

#### Tested Attention Visualizations

### Hardware Acceleration
This was run on a hardware accelerated google colab notebook with 50GB of RAM.**Using less RAM will lead to issues.** Also loading all models in the same session will lead to crashes (i.e. Mistral Instruct takes up 30GB of Memory alone).


Additionally a GPU can be used, however model interference is reasonably fast on pure gpu performance.

## Installation, Imports & Setup

### Tokens for Downloads

Without a Github token the different variant of shap cannot be loaded. Without a HGF Token llama cannot load from the huggingface hub.

This is set up for colab, alternatively the commented string variant below can be used. For this replace the string with an actual token.

*   Github [Token Info](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens)
*   Huggingface [Token Info](https://huggingface.co/docs/hub/security-tokens)

In [1]:
# grabbing tokens for repository and model access
from google.colab import userdata

gh_token = userdata.get("GITHUB_TOKEN")
hgf_token = userdata.get("HGF_TOKEN")

# gh_token="TOKEN"
# hgf_token="TOKEN"

In [ ]:
# basic installs and additional utilies (usually not needed in colab)
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install ipywidgets
!pip install ipython

# model package installs
!pip install torch
!pip install transformers
!pip install huggingface_hub
!pip install accelerate
!pip install sklearn

# bertbiz install
!pip install bertviz

In [3]:
# basic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# model imports
import torch
import transformers

# interpretability import
import bertviz
import IPython

### Setup Models

In [4]:
# setting device based on available hardware
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Device set to {device}.")

Device set to cpu.


In [26]:
# setup gpt-2 and godel model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

# gpt and godel loading function so this can be run individually
def load_gd_gpt():

    # load tokenizer and model from huggingface
    gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
    gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")

    # manage setup based on available device
    gpt_model.to(device)

    # update model config
    gpt_model.config.is_decoder = True
    gpt_model.config.max_new_tokens = 50
    gpt_model.config.do_sample = True

    # load tokenizer and model from huggingface
    gd_tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
    gd_model = AutoModelForSeq2SeqLM.from_pretrained(
        "microsoft/GODEL-v1_1-large-seq2seq"
    )

    # manage setup based on available device
    gd_model.to(device)

    # update GODEL model config
    gd_model.config.max_new_tokens = 50
    gd_model.config.do_sample = True

    return gpt_model, gpt_tokenizer, gd_model, gd_tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# mistral loading function, so this doesn't run automatically on load
def load_mistral():

    # load tokenizer and model from huggingface
    mistral_tokenizer = AutoTokenizer.from_pretrained(
        "mistralai/Mistral-7B-Instruct-v0.2"
    )
    mistral_model = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-Instruct-v0.2"
    )

    # manage setup based on available device
    mistral_model.to(device)

    # update model config
    mistral_model.config.is_decoder = True
    mistral_model.config.max_length = 50
    mistral_model.config.no_repeat_ngram_size = 2
    mistral_model.config.do_sample = True

    return mistral_model, mistral_tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# llama loading function, so this doesn't run automatically on load
def load_llama():

    # load tokenizer and model from huggingface
    llama_tokenizer = AutoTokenizer.from_pretrained(
        "meta-llama/Llama-2-7b-chat-hf", token=hgf_token
    )
    llama_model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-chat-hf", token=hgf_token
    )

    # manage setup based on available device
    llama_model.to(device)

    # update model config
    llama_model.config.is_decoder = True
    llama_model.config.max_length = 50
    llama_model.config.no_repeat_ngram_size = 2
    llama_model.config.do_sample = True

    # update tokenizer config
    llama_tokenizer.pad_token = llama_tokenizer.eos_token

    return llama_model, llama_tokenizer

**(Loading all Models in Parallel will overload the 50GB RAM)**

-> load either gpt + GODAL or Mistral or Llama2

In [27]:
# loading gpt and godel model and tokenizer
gpt_model, gpt_tokenizer, gd_model, gd_tokenizer = load_gd_gpt()

In [ ]:
# loading mistral model and tokenizer
mistral_model, mistral_tokenizer = load_mistral()

In [ ]:
# loading llama model and tokenizer
llama_model, llama_tokenizer = load_llama()

## Testing Code

### Helper Functions

In [6]:
# function to format the model reponse nicely
def format_output_text(output: list):
    # remove special tokens from list
    formatted_output = format_tokens(output)

    # start string with first list item if it is not empty
    if formatted_output[0] != "":
        output_str = formatted_output[0]
    else:
        # alternatively start with second list item
        output_str = formatted_output[1]

    # add all other list items with a space in between
    for txt in formatted_output[1:]:
        # check if the token is a punctuation mark
        if txt in [".", ",", "!", "?"]:
            # add punctuation mark without space
            output_str += txt
        # add token with space if not empty
        elif txt != "":
            output_str += " " + txt

    # return the combined string with multiple spaces removed
    return re.sub(" +", " ", output_str)


# format the tokens by removing special tokens and special characters
def format_tokens(tokens: list):
    # define special tokens to remove and initialize empty list
    special_tokens = ["[CLS]", "[SEP]", "[PAD]", "[UNK]", "[MASK]", "▁", "Ġ", "</w>"]
    updated_tokens = []

    # loop through tokens
    for t in tokens:
        # remove special token from start of token if found
        if t.startswith("▁"):
            t = t.lstrip("▁")

        # loop through special tokens and remove them if found
        for s in special_tokens:
            t = t.replace(s, "")

        # add token to list
        updated_tokens.append(t)

    # return the list of tokens
    return updated_tokens


def avg_attention(attention_values):
    attention = attention_values.cross_attentions[0][0].detach().numpy()
    return np.mean(attention, axis=0)

### Testing BertViz with GPT-2


In [ ]:
# defining test_input
test_input = "Harry is a lawyer on the east coast, his hobbies include"

In [ ]:
# creating a model and head view for GPT-2
# CREDIT: adopted from the official BERTViz documentation
## see: https://github.com/jessevig/bertviz

# imports
from transformers import utils
from bertviz import model_view, head_view

# generating a model output with attentions
inputs = gpt_tokenizer(test_input, return_tensors="pt")
out = gpt_model(**inputs, output_attentions=True)

# extracting attention from model output
attention = out["attentions"]  # Retrieve attention from model outputs
tokens = gpt_tokenizer.convert_ids_to_tokens(
    inputs["input_ids"][0]
)  # Convert input ids to token strings

# creating model and head view using BERTViz
mview = model_view(attention, tokens)
hview = head_view(attention, tokens)

In [ ]:
# creating a neuron view for GPT-2
# CREDIT: copied from the offical BERTViz documentation
## see: https://github.com/jessevig/bertviz/blob/master/notebooks/neuron_view_gpt2.ipynb

# imports
from bertviz import neuron_view as nv
from bertviz.transformers_neuron_view import GPT2Model, GPT2Tokenizer

# setting model type and version
model_type = "gpt2"
model_version = "gpt2"

# creating model and tokenizer from special BERTViz gpt classes
model = GPT2Model.from_pretrained(model_version)
tokenizer = GPT2Tokenizer.from_pretrained(model_version)

nv.show(model, model_type, tokenizer, test_input, display_mode="dark")

### Generalized Testing Functions

In [ ]:
# creating a model and head view for GODEL
# CREDIT: adopted from the official BERTViz documentation
## see: https://github.com/jessevig/bertviz

# imports
from transformers import utils
from bertviz import model_view, head_view


def hview(test_input, model, tokenizer):

    # generating a model output with attentions
    inputs = tokenizer(test_input, return_tensors="pt")
    out = model(**inputs, output_attentions=True)

    # extracting attention from model output
    attention = out["attentions"]  # Retrieve attention from model outputs
    tokens = tokenizer.convert_ids_to_tokens(
        inputs["input_ids"][0]
    )  # Convert input ids to token strings

    # creating head view using BERTViz
    view = head_view(attention, tokens, html_action="view")


def mview(test_input, model, tokenizer):

    # generating a model output with attentions
    inputs = tokenizer(test_input, return_tensors="pt")
    out = model(**inputs, output_attentions=True)

    # extracting attention from model output
    attention = out["attentions"]  # Retrieve attention from model outputs
    tokens = tokenizer.convert_ids_to_tokens(
        inputs["input_ids"][0]
    )  # Convert input ids to token strings

    # creating model and head view using BERTViz
    model_view(attention, tokens, html_action="view")

### Testing BERTViz with GODEL

In [7]:
# formatting function to formatting input for the model
# CREDIT: Adapted from official interference example on Huggingface
## see https://huggingface.co/microsoft/GODEL-v1_1-large-seq2seq
def gd_format_prompt(message: str, system_prompt: str, knowledge: str = ""):

    # adds knowledge text if not empty
    if knowledge != "":
        knowledge = "[KNOWLEDGE] " + knowledge

    # adds the message to the prompt
    prompt = f" {message}"
    # combines the entire prompt
    full_prompt = f"{system_prompt} [CONTEXT] {prompt} {knowledge}"

    # returns the formatted prompt
    return full_prompt

In [30]:
# special godel model view function because of seq2seq nature
# CREDIT: Adapted from offical BERTViz model view example for encoder-decoder models
## see https://github.com/jessevig/bertviz/blob/master/notebooks/model_view_encoder_decoder.ipynb

# imports
from bertviz import model_view

# model view function for encoder decoder models


def mview_enc_dec(test_input, model, tokenizer):
    # generating encoder and encoder inputs through tokenization
    encoder_input_ids = tokenizer(
        test_input, return_tensors="pt", add_special_tokens=True
    ).input_ids
    decoder_input_ids = model.generate(encoder_input_ids, output_attentions=True)

    # generate model output
    outputs = model(
        input_ids=encoder_input_ids,
        decoder_input_ids=decoder_input_ids,
        output_attentions=True,
    )

    # get the text for the input and output vectors
    encoder_text = format_tokens(tokenizer.convert_ids_to_tokens(encoder_input_ids[0]))
    decoder_text = format_tokens(tokenizer.convert_ids_to_tokens(decoder_input_ids[0]))

    model_view(
        encoder_attention=outputs.encoder_attentions,
        decoder_attention=outputs.decoder_attentions,
        cross_attention=outputs.cross_attentions,
        encoder_tokens=encoder_text,
        decoder_tokens=decoder_text,
        html_action="view",
    )

In [31]:
# running special model view with GODEL
mview_enc_dec(
    gd_format_prompt(
        "Does money buy happiness?",
        "Given a dialog context, you need to respond empathically.",
    ),
    gd_model,
    gd_tokenizer,
)

Output hidden; open in https://colab.research.google.com to view.

### Testing BERTViz with MistralAI

In [ ]:
# formatting function to format input for the model
# CREDIT: Inspired by offical documentation and example on Huggingface
## see https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1
def mistral_format_prompt(message: str, system_prompt: str):
    prompt = (
        f"<s>[INST] {system_prompt} [/INST] Hello, how can I assist you"
        f" today?</s>[INST] {message} [/INST]"
    )
    return prompt

In [ ]:
# running model with with mistral
mistral_test_prompt = mistral_format_prompt(
    "Does money buy happiness?",
    "Given a dialog context, you need to respond empathically.",
)
mview(mistral_test_prompt, mistra_model, mistral_tokenizer)

In [ ]:
# running head view with mistral
mistral_test_prompt = mistral_format_prompt(
    "Does money buy happiness?",
    "Given a dialog context, you need to respond empathically.",
)
hview(mistral_test_prompt, mistra_model, mistral_tokenizer)

### Testing BERTViz with Llama2

In [ ]:
# formatting function to format input for the model
# CREDIT: Adapted from Philipp Schmid
## see https://www.philschmid.de/llama-2#how-to-prompt-llama-2-chat
def llama_format_prompt(message: str, system_prompt: str):
    prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]"
    return prompt

In [ ]:
# running model view with llama 2
llama_test_prompt = mistral_format_prompt(
    "Does money buy happiness?",
    "Given a dialog context, you need to respond empathically.",
)
mview(llama_test_prompt, llama_model, llama_tokenizer)

In [ ]:
# running head view with llama 2
mistral_test_prompt = mistral_format_prompt(
    "Does money buy happiness?",
    "Given a dialog context, you need to respond empathically.",
)
hview(llama_test_prompt, llama_model, llama_tokenizer)